In [2]:
import datetime as dt
import pandas as pd
import joblib

In [3]:
df_train = pd.read_csv('training.csv', index_col='Date') # 2003 - 2013
df_val = pd.read_csv('validation.csv', index_col='Date') # 2017/08/3

MODEL_PATH = "model.pkl"
model = joblib.load(MODEL_PATH)

In [12]:
display(df_train.columns)

df_val = pd.read_csv('validation.csv', index_col='Date')
#display(df_val)
df_val.drop(columns=['Unix', 'Quarterly Revenue Growth', 'Gross Profit', 'Float', '% Held by Insiders', '% Held by Institutions', 'Shares Short',
       'Short Ratio', 'Short % of Float', 'Shares Short (prior month', 'Operating Cash Flow', 'Quarterly Earnings Growth',
       'Levered Free Cash Flow'], inplace=True)

df1_columns = [col for i, col in enumerate(df_train.columns) if i not in []]
df2_columns = [col for i, col in enumerate(df_val.columns) if i not in []]

for col1, col2 in zip(df1_columns, df2_columns):
    print(f"df1: {col1} df2: {col2}")

Index(['Ticker', 'Price', 'stock_p_change', 'SP500', 'SP500_p_change',
       'Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'EBITDA', 'Net Income Avl to Common', 'Diluted EPS', 'Total Cash',
       'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding'],
      dtype='object')

df1: Ticker df2: Ticker
df1: Price df2: Price
df1: stock_p_change df2: stock_p_change
df1: SP500 df2: SP500
df1: SP500_p_change df2: SP500_p_change
df1: Market Cap df2: Market Cap
df1: Enterprise Value df2: Enterprise Value
df1: Trailing P/E df2: Trailing P/E
df1: Forward P/E df2: Forward P/E
df1: PEG Ratio df2: PEG Ratio
df1: Price/Sales df2: Price/Sales
df1: Price/Book df2: Price/Book
df1: Enterprise Value/Revenue df2: Enterprise Value/Revenue
df1: Enterprise Value/EBITDA df2: Enterprise Value/EBITDA
df1: Profit Margin df2: Profit Margin
df1: Operating Margin df2: Operating Margin
df1: Return on Assets df2: Return on Assets
df1: Return on Equity df2: Return on Equity
df1: Revenue df2: Revenue
df1: Revenue Per Share df2: Revenue Per Share
df1: EBITDA df2: EBITDA
df1: Net Income Avl to Common df2: Net Income Avl to Common
df1: Diluted EPS df2: Diluted EPS
df1: Total Cash df2: Total Cash
df1: Total Cash Per Share df2: Total Cash Per Share
df1: Total Debt df2: Total Debt
df1: Total Debt/

In [15]:
import numpy as np

def prepare_data(df):

    df.dropna(axis=0, how="any", inplace=True)

    features = df.columns[5:]
    display(features)
    display(features.shape)
    x = df[features]

    #scaler = StandardScaler()
    # Fit and transform the data
    #x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

    return x

X = prepare_data(df_val)


Index(['Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'EBITDA', 'Net Income Avl to Common', 'Diluted EPS', 'Total Cash',
       'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding'],
      dtype='object')

(29,)

In [16]:
predictions = model.predict(X)

In [25]:
preds = pd.Series(predictions, name='Predictions')
#X_original = pd.DataFrame(scaler.inverse_transform(X), columns=X.columns)
merged = pd.concat([df_val.reset_index(drop=True), preds.reset_index(drop=True)], axis=1)

stock_symbols = merged[merged['Predictions'] == True]['Ticker'].to_list()
stock_symbols

['CAH',
 'LSI',
 'CNX',
 'JBL',
 'BAX',
 'FDX',
 'SWK',
 'HTZ',
 'RRC',
 'NBL',
 'LH',
 'SNA',
 'GT',
 'CRM',
 'BIIB',
 'ICE',
 'QDEL',
 'DDS',
 'AIZ',
 'GNW',
 'DNR',
 'BRK-B',
 'RL',
 'SFLY',
 'WAT',
 'PBI',
 'BLK',
 'DLX',
 'FB',
 'KR',
 'CI',
 'AMD',
 'PSA',
 'AMP',
 'BBBY',
 'APD']

In [24]:
start = dt.datetime(2017, 8, 3)
end = dt.datetime(2018, 8, 3)
results = []
average = 0
import yfinance as yf

for stock in stock_symbols:
    df = yf.download(stock, start=start, end=end)['Adj Close']
    if len(df) > 0:
        pct_change = df[-1] / df[0]
        results.append((stock, pct_change))
        average += pct_change

for stock, pct_change in results:
    print(f"Stock: {stock}, Percentage Change: {(pct_change-1):.2%}")
print(f"{(average/len(results) - 1) * 100} %")

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_6088\1143581877.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['LSI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_6088\1143581877.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]
[*********************100%%*********

Stock: CAH, Percentage Change: -18.99%
Stock: CNX, Percentage Change: 12.62%
Stock: JBL, Percentage Change: -10.09%
Stock: BAX, Percentage Change: 31.37%
Stock: FDX, Percentage Change: 15.51%
Stock: SWK, Percentage Change: 5.93%
Stock: RRC, Percentage Change: -2.16%
Stock: LH, Percentage Change: 7.96%
Stock: SNA, Percentage Change: 24.50%
Stock: GT, Percentage Change: -23.18%
Stock: CRM, Percentage Change: 74.06%
Stock: BIIB, Percentage Change: 20.73%
Stock: ICE, Percentage Change: 16.68%
Stock: QDEL, Percentage Change: 104.25%
Stock: DDS, Percentage Change: 5.94%
Stock: AIZ, Percentage Change: 6.38%
Stock: GNW, Percentage Change: 23.72%
Stock: BRK-B, Percentage Change: 20.28%
Stock: RL, Percentage Change: 75.13%
Stock: WAT, Percentage Change: 8.87%
Stock: PBI, Percentage Change: -40.26%
Stock: BLK, Percentage Change: 15.23%
Stock: DLX, Percentage Change: -21.32%
Stock: KR, Percentage Change: 23.95%
Stock: CI, Percentage Change: 19.65%
Stock: AMD, Percentage Change: 119.18%
Stock: PSA,


C:\Users\Bogdan\AppData\Local\Temp\ipykernel_6088\1143581877.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]
